In [ ]:
import tensorflow as tf 
from tensorflow import keras 
from keras import layers 
from dataclasses import dataclass # có chức năng tạo các lớp dữ liệu data 
import numpy as np 
import  pandas as pd 
import glob 
import re 
from pprint import pprint #Hàm pprint trong python là một hàm
# dùng để in các cấu trúc dữ liệu trong python 

Thiết lập cấu hình 

In [ ]:
@dataclass
class  Config:
    MAX_LEN = 256 
    BATCH_SIZE = 32
    LR = 0.001
    VOCAB_SIZE = 30000
    EMBED_DIM = 128 
    NUM_HEAD = 8 # USE IN BERT MODEL 
    FF_DIM  = 128 # USE IN BERT MODEL 
    NUM_LAYERS = 1 

config = Config()

Tải dữ liệu

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

In [ ]:
# Xây dựng hàm lấy danh sách các văn bản từ file 
def get_text_list_from_file(files):
    # tạo một danh sách để chứa các text 
    text_list = []
    # Duyệt qua mỗi phần tử trong danh sách 
    for name in files:
        # sử dụng dòng lệnh with open(name) để mở các file và gán nó cho biến f 
        with open(name) as f :
            # duyệt qua mỗi dòng văn bản thông qua các tệp file mở 
            for line in f:
                # add các dòng vào danh sách 
                text_list.append(line)
    return text_list


# Xây dựng hàm đọc tệp dữ liệu từ các dòng văn bản trong thư mục 
# và trả về 1 bảng dữ liệu với 2 cột là review và sentiment
def get_data_from_text_files(folder_name):
    # sử dụng hàm glob để lấy danh sách các tệp văn bản trong thư mục con pos 
    # của thư mục folder_name và các tệp này có định dạng  aclImdb/folder_name/pos/*.txt
    pos_files = glob.glob("aclImdb/" + folder_name + "/pos/*.txt")
    # Tiến hành đọc và lấy các dòng dữ liệu từ file 
    pos_texts = get_text_list_from_file(pos_files) 
    # thực hiện tương tự với định dạng thư mục aclImdb/folder_name/neg/*.txt
    neg_files = glob.glob("aclImdb/" + folder_name + "/neg/*.txt")
    neg_texts = get_text_list_from_file(neg_files)

    # Tiến hành xây dựng dataFrame với 2 cột review và sentiment 
    df = pd.DataFrame (
        {
            'review' : pos_texts + neg_files, # là số lượng phần tử trong cột 
            # với cột sentiment ta biến đổi dạng 0 và 1 
            'sentiment': [0] * len(pos_texts) + [1] * len(neg_texts), 
            
        }
    )
    # sử dụng hàm sample để lấy mẫu ngẫu nhiên với len (df) là độ dài của các cột 
    # kết quả là một dataFrame mới có cùng kính thước nhưng số lượng hàng nhưng 
    # số lượng hàng bị xáo trộn 
    # Sử dụng hàm reset_index để đặt lại ví trí ccho các hàng sau khi bị trộn 
    df = df.sample(len(df)).reset_index(drop=True)
    return df 


# Xây dựng dữ liệu train vè test 
train_df = get_data_from_text_files('train')
test_df = get_data_from_text_files('test')
all_data = train_df.append(test_df)


Prepare the Data

In [ ]:
# Xây dụng hàm tùy chỉnh tiêu chuẩn hóa dữ liệu 
# Có chức năng thay thế loại bỏ xuống ròng và biến đổi thường văn bản 
def custom_standardization(input_data):
    # biến đổi dữ liệu thành chữ thường 
    lowercase = tf.strings.lower(input_data)
    # xây dựng phương thức xử lý thay thế chính quy regex để thay thế 
    # biểu thức chính quy <br /> trong html thành các khoảng trắng 
    tripped_html = tf.strings.regex_replace(lowercase , "<br />" , " ")
    # xây dựng phương thức trả về bằng cách sử dụng biểu thức thay thế regex
    # phương thức sẽ tìm kiếm các ký tự tromng danh sách và thay thế nó 
    # Sử dụng re.escape để thoát khỏi các kỹ tự có ý nghĩa đặc biệt trong biểu thức 
    # chính quy 
    return tf.strings.regex_replace(
        tripped_html , "[%s]" % re.escape("!#$%&'()*+,-./:;<=>?@\^_`{|}~"),""
    )

# Xây dựng hàm Thực hiện vector hóa dữ liệu 
def get_vectorize_layer(text, vocab_size , max_seq , special_tokens=['[MASK]']):
    """Xây dựng Vector hóa văn bản 

    Args:
        Text (list) : Danh sách của chuỗi i.e văn bản đầu vào . 
        Vocab (int) : Kích thước của tập từ điển . 
        Max seq length (int) : Độ dài mô hình nhận được .
        Special_tokens (list , optional) : Danh sách các mã thông báo đặc biệt . Xác định [MASK]

    Return 
        layers.Layer: Return TextVectorization keras.Layer
    """
    # Thiết lập phương thức vector háo token int 
    vectorize_layer = keras.layers.TextVectorization(
            # truyền vào số lượng token tối đa  = vocab_size 
            max_tokens=vocab_size, 
            # Dạng biến đổi = int for each token
            output_mode='int',
            # Tiêu chuẩn cho các tokens , loại bỏ đi cá ký tự đầu vào 
            standardize=custom_standardization,
            # kích thước đầu ra 
            output_sequence_length=max_seq,
    )
    # biến đổi phù hợp dữ liệu text để đưa vào vector hóa
    vectorize_layer.adapt(text)
    
    # Xử lý và trèn mặt ạn cho câu 

    # Xay dựng tập từ vựng với get_vocabulary () trả về 1 danh sách từ vựng 
    # Trong phương thức Tokenizer of text với tần suất xuất hiện giảm dần 
    vocab = vectorize_layer.get_vocabulary()
    
    # Thực hiện quy định cho bộ từ vựng 
    # Tiến hành xây dựng bộ từ vựng mới bằng cachs loại bỏ đi 2 phần từ đầu 
    # và loại bỏ các tokens đặc biệt sau đó sử dụng mặt nạ mask để bù vào số phần tử 
    # bị loại bỏ cho tập từ vựng nhằm giữ cho tính đa dạng tập từ vựng 
    vocab = vocab[2 : vocab - len(special_tokens)] + ['[MASK]'] 
    # Cập nhật lại bộ từ vụng 
    vectorize_layer.set_vocabulary(vocab)


# biến đổi trên tập dữ liệu 
vectorize_layer = get_vectorize_layer(
    text= all_data.review.values.tolist(), 
    vocab_size = config.VOCAB_SIZE, 
    max_len  = config.MAX_LEN, 
    special_tokens=['[mask]'],
)

# Nhận id mã thông báo mặt ạn cho mô hình ngôn ngữ 
# Biến đổi mặt nạ trong tập từ điển thành mảng numpy int như các tokens 

mask_token_id = vectorize_layer(["[mask]"]).numpy()[0][0]

# Xây dựng hàm thực hiện mã hóa dữ liệu 
# sử dụng hàm vectorize_layer được tạo truơc 
def encode(texts):
    encoded_texts = vectorize_layer(texts)
    # sau khi thực hiện mã hóa tokens ta chuyển các vector dạng numpy 
    # để được các ma trận dngj int 
    return encoded_texts.numpy()


# Xây dựng lớp mặt nạ cho các token (int) 
# Lớp này sẽ thực hiện che đi 15 % số lượng token từ chuỗi token đầu vào 
def get_masked_input_labels(encoded_texts):
    # khởi tạo mặt nạ đầu vào với kích thước = input 
    # che đi 15 % số phần tử 
    inp_mask = np.random.rand(*encoded_texts.shape) < 0.15
    # Không tre những tokens có giá trị nhỏ hơn 2 
    # tức là những giá trị <= 2 bị bỏ qua 
    inp_mask[encoded_texts <= 2 ] = False 
    # tạo ma trận labels có shape [input] và input fully = -1 
    # Trong quá trình huấn luyện các giá trị = -1 bị bỏ qua 
    labels = -1 * np.ones(encoded_texts.shape, dtype=int)

    # Ta đặt nhãn cho những tokens bị che đi bằng - 1 bằng cách gán 
    # giá trị của chúng trong encode_text vào vị trí tương ứng trong ma trận labels 
    # ma trận labels[inp_mask] có các giá trị = -1 
    # và ma trận encoded_text[inp_mask]  có các gia strị khác trừ 1

    # kết quả ma trận labels những phần tử bị che mặt nạ trong ma trận 
    # được gán giá trị tương tự với vị trí của nó trong labels[inp_mask] = -1 
    labels[inp_mask] = encoded_texts[inp_mask]
    
    # Chuẩn bị dữ liệu 
    encoded_texts_masked = np.copy(encoded_texts)

    # Tập dữ liệu đầu vào là [mask] là các token cuối cùng cho 90 % số mã thông báo  
    # Điều này có nghĩa là giữ nguyên 10 % 
    # và trong số này không tính những token đặc biệt 
    inp_mask_2mask = inp_mask & (np.random.rand(*encoded_texts.shape) < 0.9)
    # Sau đó thay thế các token được che mặt nạ bằng token Mask 
    encoded_texts_masked [ inp_mask_2mask ] =  mask_token_id  

    # Tạo ra bộ mặt nạ khác để chọn 10% từ số lượng 90% token mask 
    inp_mask_2random = inp_mask_2mask &  (np.random.rand(*encoded_texts.shape)< 1 / 9)
    # Sau đó thay thế các tokens được chọn bởi mặt nạ inp_mask_2random = các tokens ngẫu nhiên
    # khác trừ những token đặc biệt khác trừ các token đặc biệt 
    encoded_texts_masked[inp_mask_2mask] = np.random.randint(
        3 , mask_token_id , inp_mask_2random.sum)
    
    # Chuẩn bị sample_weights để chuyển sang phương thức weights 
    sample_weights = np.ones(labels.shape)
    # Gán cho nhãn bằng -1 tức là các tokens đặc biệt và trọng số mâũ = 0 để
    # qua trong quá trình huấn luyện 
    sample_weights[labels == - 1] = 0

    # Xây dựng nhãn y cho bộ tokens mã hóa 
    # gán  = encoded_text 
    y_labels = np.copy(encoded_texts)

    return encoded_texts_masked , y_labels , sample_weights 


# Tách dữ liệu huấn luyện và dữ liệu đào tạo 
x_train = encode(train_df.review.values)
y_train = train_df.sentiment.values 

# Từ x , y train t gộp nó thành một tensor data
train_classifier_ds = (
    tf.data.Dataset.from_tensor_slices((x_train , y_train))
    .shuffle(1000)
    .batch(config.BATCH_SIZE)
)

# 25000 mẫu dữ liệu cho thử nghiệm 
x_test = encode(test_df.review.values)
y_test = test_df.sentiment.values 

test_classifier_ds =  (
    tf.data.Dataset.from_tensor_slices((x_test , y_test))
    .batch(config.BATCH_SIZE)
)


# Xây dựng đầu vào từ đầu đến cuối cho mô hình ( sẽ được sử dụng ở cuối cùng ) 
# Xử dụng hàm tensor_slices để tạo một dataset từ các tensor 
test_raw_classifier_ds =  tf.data.Dataset.from_tensor_slices(
    (test_df.review.values, y_test)
)

# Chuẩn bị dữ liệu cho mô hình mặt nạ ngôn ngữ 
all_review = encode(all_data.review.values)
x_masked_train , y_masked_train , sample_weights = get_masked_input_labels(
    all_review 
)

mlm_ds = tf.data.Dataset.from_tensor_slices(
    (x_masked_train , y_masked_train, sample_weights )
)
mlm_ds = mlm_ds.shuffle(1000).batch(config.BATCH_SIZE)


Create Bert model (Pretraining Model)  for masked language model 

In [ ]:
def bert_model(query , key, value, i):
    # Multi head self-attention 
    attention_output = layers.MultiHeadAttention(
        num_heads=config.NUM_HEAD , 
        key_dim=config.NUM_HEAD, 
        key_dim= config.EMBED_DIM // config.NUM_HEAD, 
        name='encoder_{}/multiheadattention'.format(i), 

    )(query , key , value)
    # Add and norm layer for multi head attention output
    attention_output = layers.Dropout(0.1, name='encoder_{}/attn_dropout'.format(i))(
        attention_output
    )
    # ADD layerNormalization for attention
    attention_output = layers.LayerNormalization(
        epsilon=1e-6, name='encoder_{}/att_layernormalization'.format(i)
    )(query + attention_output)

    # Feed-forward layer
    ffn = keras.Sequential(
        [
            layers.Dense(config.FF_DIM , activation='relu'),
            layers.Dense(config.EMBED_DIM),
        ],
        name ='encoder_{}/ffn'.format(i),
    )
    # ffn layer
    ffn_output = ffn(attention_output)
    ffn_output = layers.Dense(0.1 , name='encoder_{}/ffn_dropout'.format(i))(
        ffn_output
    )
    # sequence output = attention_output + ffn_output
    sequence_output = layers.LayerNormalization(epsilon=1e-6,
                        name='encoder_{}/ffn_layernormalization'.format(i)
        )(attention_output + ffn_output)
    return sequence_output


# Xây dụng hàm tạo ma trận positional encoding 
def get_pos_encoding_matrix(max_len, d_emb):
    pos_enc = np.array(
        [
            # theo công thức pos / 10000^2 *j // d / emb
            # công thức này là công thức tính toán vị trí trong transformer gốc 
            [pos / np.power(10000, 2 *(j // 2) /d_emb) for j in range(d_emb)    ]
            if pos != 0 
            else np.zeros(d_emb)
            for pos in range(max_len)
        ]
    )
    # T thay thế các giá trị ở chiều chằn và lẻ 
    # trường hợp chẵn ta sử dụng hàm sin
    pos_enc[1:, 0 :: 2] = np.sin(pos_enc[1:, 0:: 2])
    # Trường hợp lẻ ta sử dụng cos function 
    pos_enc[1:, 1:: 2] = np.cos(pos_enc[1:, 1:: 2])
    # Cuối cùng trả về vị trí cho các tokens 

# xây dựng hàm loss và phương thức loss tracker để cập nhật số liệu 
loss_fn = keras.losses.SparseCategoricalCrossentropy(
    reduction=tf.keras.losses.Reduction.NONE
)
loss_tracker = tf.keras.metrics.Mean(name='loss')


# Xây dựng mô hình mặt nạ ngôn ngữ 
class MaskedLanguageModel(keras.Model):
    def train_step(self, inputs):
        if len(inputs) == 3 :
            features, labels , sample_weight = inputs
        else:
            features , labels = inputs 
            sample_weight = None
        
        with tf.GradientTape() as tape:
            predictions = self(features, training=True)
            loss = loss_fn(labels, predictions, sample_weight=sample_weight)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss , trainable_vars)

        # update Weights 
        self.optimizer.apply_gradients(zip(gradients , trainable_vars))
        # Tính toán và cập nhật số liệu 
        loss_tracker.update_state(loss, sample_weight=sample_weight)

        # TRả lại tên chỉ số ánh xạ chính tả và chỉ số hiện tại 
        return {'loss': loss_tracker.result()}
    

    @ property 
    def metrics(self):
        return [loss_tracker]


#  Khởi tạo mô hình mặt nạ ngôn ngữ với bert 

def create_masked_language_bert_model():
    # Create input layers shape = max_sequen_length data type = int64 
    inputs = layers.Input((config.MAX_LEN,), dtype=tf.int64)

    # Create nhúng từ + position embedding 
    word_embedding = layers.Embedding(
        input_dim = config.VOCAB_SIZE ,
        output_dim = config.EMBED_DIM , name='word_embedding'
    )
    position_embedding = layers.Embedding (
        config.MAX_LEN,
        output_dim=config.EMBED_DIM, 
        weights = [get_pos_encoding_matrix(max_len=config.MAX_LEN,d_emb=config.EMBED_DIM)], 
        name='position_embedding'
    )(tf.range(start=0 , limit=config.MAX_LEN, delta=1))
    embeddings = word_embedding +  position_embedding

    encoder_output = embeddings 

    # ADD 8 layers bert modeule
    for i in range(config.NUM_LAYERS):
        encoder_output = bert_model(encoder_output, encoder_output, encoder_output,i)

    # Create mlm_output
    mlm_output = layers.Dense(units=config.VOCAB_SIZE , name='mlm_cls',
                 activation='softmax')(encoder_output)
    
    mlm_model = MaskedLanguageModel(inputs , mlm_output , name='masked_bert_model')

    optimizer = keras.optimizers.Adam(learning_rate=config.LR)
    mlm_model.compile(optimizer=optimizer)
    return mlm_model 


# tạo ra 2 bộ từ điển mới 
id2token = dict(enumerate(vectorize_layer.get_vocabulary()))
token2id = {y : x for x , y in id2token.items() }

class MaskedTextGenerator(keras.callbacks.Callback):
    def __init__(self, sample_tokens, top_k=5):
        self.sample_tokens = sample_tokens 
        self.k = top_k

    def decode(self,tokens):
        return " ".join([id2token[t] for t in tokens if t!= 0])
    
    def convert_ids_to_tokens(self, id):
        return id2token[id]
    
    def on_epoch_end(self, epoch , logs=None):
        # dự đoán các token có thể điền vào vị trí của token bị che đi 
        # Kết quả là một ma trận chứa xác xuất cho mỗi token có thể 
        prediction = self.model.predict(self.sample_tokens)
        # tìm ra chỉ số của token che dấu trong câu chỉ số này được lưu trong 
        # ma trận mask_token_id là các chỉ số cột một cột = 1 token
        masked_idex = np.where(self.sample_tokens == mask_token_id)
        # ta chỉ quan tâm đến chỉ số hàng nên ta để là chiều thứ 2 
        masked_idex = masked_idex[1]
        # lấy ra ma trận có xác xuất cao nhất cho token bị che giấu
        # shape = [1, vocabulary] mỗi phần tử trong ma trận là xác xuất của 1 token 
        # có thể được điền vaò vị trí bị che giấu 
        masked_prediction = prediction[0][masked_idex]

        # sắp xếp các chỉ số xác xuất tokens từ cao xuống thấp 
        top_indices = masked_prediction[0].argsort()[-self.k :][:: -1]
        # lấy ra các giá trị tương ứng với xác xuất cao nhất.
        values = masked_prediction[0][top_indices]

        # Duyệt qua ma trận kết quả chứa xác xuất 
        for i in range(len(top_indices)):
            # lấy ra từng xác xuất của token tương ứng
            p = top_indices[i]
            # giá trị tương ứng với xác xuất 
            v = values[i]
            # tạo một bản sao của câu ban đầu 
            # tránh thay đổi giá trị của câu nguyên mẫu gốc khi thực hiện 
            # các thao tác 
            tokens = np.copy(self.sample_tokens[0])
            # thay thế các token bị tre giấu bằng xác xuất dự đoán p 
            # t đặt chỉ số hàng để xác xuất có thể đặt vào vị trí token bị che đi 
            tokens[masked_idex[0]] =  p 
            # Thực hiện biến đổi kết quả 
            result = {
                # biến đổi mẫu đầu vào thành ma trận số
                "input_text" : self.decode(self.sample_tokens[0].numpy()),
                "prediction" : self.decode(tokens),
                # xác xuất của token dự đoán p được lấy từ giá trị v
                "probability": v,
                # chuyển đổi xác xuất p và v tương ứng vói nó thành chuỗi 
                "predicted mask token": self.convert_ids_to_tokens(p)

            }
            pprint(result)


sample_tokens = vectorize_layer(['I have watched this [mask] , and it was awesome'])
generator_callbacks = MaskedTextGenerator(sample_tokens.numpy())
        

In [ ]:
bert_masked_model = create_masked_language_bert_model()
bert_masked_model.summary()

Train and save

In [ ]:
bert_masked_model.fit(mlm_ds , epochs=5 ,callbacks=[generator_callbacks])
bert_masked_model.save('bert_mlm_imdb.h5')

Fine-tune a sentiment classification model 

In [ ]:
# Load Pretrained bert model 
mlm_model = keras.models.load_model(
    "bert_mlm_imdb.h5" , custom_objects= {'MaskedLanguageModel': MaskedLanguageModel}
)
pretrained_bert_model = tf.keras.Model(
    mlm_model.input , mlm_model.get_layer('encoder_0/ffn_layernormalization').output
)

# Đóng băng mô hình 
pretrained_bert_model.trainable = False 

def create_classifier_bert_model():
    inputs = layers.Input((config.MAX_LEN,), dtype=tf.int64)
    sequence_output = pretrained_bert_model(inputs)
    pooled_output = layers.GlobalMaxPooling1D()(sequence_output)

    hidden_layer= layers.Dense(units= 64, activation='relu')(pooled_output)
    outputs = layers.Dense(1, activation='sigmoid')(hidden_layer)

    classifier_model = keras.Model(inputs=inputs , outputs=outputs, name='classification')
    optimizer = keras.optimizers.Adam()

    classifier_model.compile(optimizer=optimizer, loss='binary_crossentropy'
                             , metrics=['accuracy'])
    return classifier_model



classifier_model = create_classifier_bert_model()
classifier_model.summary()


In [ ]:
# Train classifier with fronze bert stage
classifier_model.fit(
    train_classifier_ds ,
    epochs=5, 
    validation_data=test_classifier_ds,
)

# Unfreeze the Bert model for fine-tuning 
pretrained_bert_model.trainable= True
optimizer = keras.optimizers.Adam()
classifier_model.compile(
    optimizer=optimizer , loss='bonary_crossentropy', metrics=['accuracy']
)
classifier_model.fit(
    train_classifier_ds , 
    epochs=5 , 
    validation_data=test_classifier_ds,
)

Create end-to-end model and evaluate it 

In [ ]:
def get_end_to_end(model):
    inputs_string = layers.Input(shape=(1,), stype='string')
    # thực hiện bert Tokenizer 
    indices = vectorize_layer(inputs_string)
    outputs = model(indices)
    # End-to-end model 
    end_to_end_model = keras.Model(inputs=inputs_string , outputs=outputs,
                        name='end_to_end_model')
    
    # optimizers 
    optimizer = keras.optimizers.Adam(learning_rate=config.LR)
    end_to_end_model.compile(
        optimizer=optimizer , loss= 'binary_crossentropy', 
        metrics=['accuracy']
    )
    return end_to_end_model


end_to_end_classification_model = get_end_to_end(classifier_model)
end_to_end_classification_model.evaluate()
